In [1]:
import csv
import numpy as np
from sklearn import linear_model
from sklearn import preprocessing

In [2]:
def get_river_dataset(fname, pr_list=None, y_name='h_max'):
    pr_arr = []
    y_arr = []
    with open(fname, newline='') as f:
        reader = csv.DictReader(f, delimiter=';')
        for row in reader:
            pr_arr_row = []
            for col in pr_list:
                pr_arr_row.append(row[col])
                #print(f'{col}: {row[col]}', end='; ')
            pr_arr.append(pr_arr_row)
            y_arr.append(row[y_name])
    X = np.asarray(pr_arr, dtype=np.float32)
    y = np.asarray(y_arr, dtype=np.float32)
    return X, y

#### Сумма, средний, высший, низший уровни

In [3]:
def get_sum(h_max):
    return np.sum(h_max)
    
def get_avg(h_max):
    return np.mean(h_max)
    
def get_max(h_max):
    return np.amax(h_max)
    
def get_min(h_max):
    return np.amin(h_max)

#### Среднеквадратическая погрешность прогноза S

In [4]:
def get_s(h_max, h_forecast=None):
    # Среднеквадратическая погрешность прогноза
    n = h_max.shape[0]
    sqr_diff = np.sum((h_max - h_forecast) ** 2) / (n - 1)
    std = sqr_diff ** 0.5
    return std    

#### Среднеквадратическое отклонение sigma

In [5]:
def get_sigma(h_max):
    # Среднеквадратическая погрешность климатическая.
    # Рассчитывается только по всей совокупности данных.
    return np.std(h_max, ddof=1)

In [6]:
def get_hmax_avg(h_max):
    # Среднее значение h_max.
    # Рассчитывается только по всей совокупности данных.
    return np.mean(h_max)

#### Допустимая погрешность прогноза delta_dop

In [7]:
def get_delta_dop(sigma):
    return 0.674 * sigma

#### Критерий эффективности метода прогнозирования климатический S/sigma

In [8]:
def get_criterion(s, sigma):
    #print(s / sigma)
    return s / sigma

#### Климатическая обеспеченность Pk

In [9]:
def get_pk(h_max, h_max_avg, delta_dop):
    #avg_level = np.mean(h_max)
    diff = np.abs(h_max - h_max_avg)
    trusted_values = diff[diff <= delta_dop]
    m = trusted_values.shape[0]
    n = h_max.shape[0]
    return m / n * 100.00

#### Обеспеченность метода (оправдываемость) Pm

In [10]:
def get_pm(h_max, h_forecast, delta_dop):
    diff = np.abs(h_max - h_forecast) / delta_dop
    trusted_values = diff[diff <= 1.0]
    m = trusted_values.shape[0]
    n = h_max.shape[0]
    #print(m, n)
    return m / n * 100.00

#### Корреляционное отношение

In [11]:
def get_correlation_ratio(criterion):
    c_1 = (1 - criterion ** 2)
    ro = c_1 ** 0.5 if c_1 > 0 else 0
    return ro

#### Вероятная ошибка прогноза S'

In [12]:
def get_forecast_error(s):
    return 0.674 * s

#### Ошибки климатического/природного прогноза для каждого года delta50

In [13]:
def get_delta50(h_max, delta_dop, h_max_avg=None, h_max_forecast=None):
    if h_max_forecast is None:
        # delta50 климатическая
        return np.abs(h_max - h_max_avg) / delta_dop
    else:
        # delta50 прогноза
        return np.abs(h_max - h_max_forecast) / delta_dop
  

#### Функция записи в csv файл

In [14]:
import csv
def write_dataset_csv(dataset, filename, fieldnames):
    with open(f'results/{filename}.csv', 'w', newline='') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames, delimiter=';', extrasaction='ignore')
        writer.writeheader()
        writer.writerows(dataset)


#### Функция разделения набора данных на тренировочный и тестовый

In [15]:
def train_test_split(X, y, n_test):
    X_train = X[:-n_test]
    y_train = y[:-n_test]
    X_test = X[-n_test:]
    y_test = y[-n_test:]
    return X_train, y_train, X_test, y_test

#### Функция получения датасетов

In [16]:
def get_datasets():
    datasets = {
        'Неман-Белица': 'Неман',
        'Неман-Гродно': 'Неман',
    }
    return datasets

#### Функция получения списка предикторов по названию датасета

In [17]:
def get_predictors(dataset_name):

    datasets = get_datasets()   
    
    predictors_lists = {
        'Неман': ['s_2802', 's_max', 'h', 'x', 'x1', 'x2', 'x3', 'x4', 'xs'],
    }
    return predictors_lists[datasets[dataset_name]]
    

#### Функция формирования проверочных прогнозов

In [18]:
def verify_forecast(dataset_name, model, n_test=9):

    ds_dir = 'data' # В константы

    #datasets = get_datasets()
    pr_list = get_predictors(dataset_name)
    pr_list = ['year'] + pr_list
    
    fieldnames = [
        'Год',
        'Hmax фактический', 'Hф-Hср', '(Hф-Hср)^2', 'Погрешность климатических прогнозов в долях от допустимой погрешности',
        'Hmax прогнозный', 'Hф-Hп', '(Hф-Hп)^2', 'Погрешность проверочных прогнозов в долях от допустимой погрешности',
    ]

    X, y = get_river_dataset(f'{ds_dir}/{dataset_name}.csv', pr_list=pr_list, y_name='h_max')

    if n_test is not None and n_test != 0:
        _, _, X_test, y_test = train_test_split(X, y, n_test)
    else:
        X_test = X
        y_test = y

    # Выделение первой колонки (года) из набора предикторов
    years = X_test[:, 0]
    X_test = X_test[:, 1:]
    
    # if n_test is not None:
    #     X = h_max[-9:]
    # else:
    #     X = h_max
    
    # Forecast
    h_max_forecast = model.predict(X_test)
    
    # Hсредний
    h_max_avg = np.mean(y)

    # H - Hсредний
    diff_fact = y_test - h_max_avg

    # (H - Hсредний) в квадрате
    diff_fact_sqr = diff_fact ** 2

    # Погрешность климатических прогнозов в долях от допустимой погрешности
    delta_dop = get_delta_dop(get_sigma(y))
    error_climate = get_delta50(y_test, delta_dop, h_max_avg=h_max_avg)

    # H - Hпрогнозный
    diff_forecast = y_test - h_max_forecast

    # (H - Hпрогнозный) в квадрате
    diff_forecast_sqr = diff_forecast ** 2       

    # Погрешность проверочных прогнозов в долях от допустимой погрешности
    error_forecast = get_delta50(y_test, delta_dop, h_max_forecast=h_max_forecast)

    # Конкатенация массивов
    att_tuple = (years, y_test, diff_fact, diff_fact_sqr, error_climate, h_max_forecast, diff_forecast, diff_forecast_sqr, error_forecast)
    arr = np.column_stack(att_tuple)
    arr = arr.tolist()
    
    # Запись проверочного прогноза в csv файл
    with open(f'results/{dataset_name}-проверочный.csv', 'w', newline='') as csvfile:
        writer = csv.writer(csvfile, delimiter=';')
        writer.writerow(fieldnames)
        writer.writerows(arr)
        # for i in arr:
        #     writer.writerow(dataset)
        
        
        

#### Функция обучения и оценки моделей

In [19]:
def compare_models(validation=False, n_test=9, norms=False):
    from sklearn.linear_model import LinearRegression
    from sklearn.linear_model import Ridge
    from sklearn.linear_model import Lasso
    from sklearn.linear_model import ElasticNet, ElasticNetCV
    from sklearn.linear_model import Lars, LarsCV
    from sklearn.linear_model import LassoLars

    from sklearn.gaussian_process import GaussianProcessRegressor
    from sklearn.gaussian_process.kernels import DotProduct, WhiteKernel, RBF
    
    
    ds_dir = 'data' # В константы
    
    names = [
        'LinearRegression',
        'Ridge',
        'Lasso',
        'ElasticNet',
        'ElasticNetCV',
        'LassoLars',
        'Lars1',
        'Lars2',
        'Lars3',
        'Lars4',
        'Lars5',
        'Lars6',
        'Lars7',
        'Lars8',
        'LarsCV',
        
        #'GaussianProcessRegressor',
        
    ]

    regressors = [
        LinearRegression(),
        Ridge(alpha=10),
        Lasso(alpha=100.0),
        ElasticNet(alpha=2.0, l1_ratio=0.1),
        ElasticNetCV(l1_ratio=[.1, .5, .7, .9, .95, .99, 1], eps=0.01, n_alphas=1000),
        LassoLars(alpha=1.0),
        Lars(n_nonzero_coefs=1),
        Lars(n_nonzero_coefs=2),
        Lars(n_nonzero_coefs=3),
        Lars(n_nonzero_coefs=4),
        Lars(n_nonzero_coefs=5),
        Lars(n_nonzero_coefs=6),
        Lars(n_nonzero_coefs=7),
        Lars(n_nonzero_coefs=8),
        LarsCV(max_iter=5000, max_n_alphas=10000, cv=3),

        #GaussianProcessRegressor(kernel=RBF(length_scale=1.1) + WhiteKernel() + DotProduct(), random_state=0)
    ]

    datasets = get_datasets()
    # datasets = [
    #     ('Неман-Белица', 'Неман'),
    #     ('Неман-Гродно', 'Неман')
    # ]

    
    # predictors_lists = {
    #     'Неман': ['s_2802', 's_max', 'h', 'x', 'x1', 'x2', 'x3', 'x4', 'xs'],
    # }

    norms = {
        
    }

    fieldnames = ['Predictors', 'Equations', 'Method', 'Criterion', 'Correlation', 'Pm']

    # datasets_result = {
    #     "hydropost_0": [
    #         { model_row }
    #         { model_row }
    #     ],
    #     ...,
    #     "hydropost_n": [
    #         { model_row }
    #         { model_row }
    #     ],
    # }
    
    
    # Итерация по датасетам
    datasets_result = dict()
    for ds in datasets:
        datasets_result[ds] = []
        # one_dataset_row = dict()
        
        pr_list = get_predictors(ds)
        
        X, y = get_river_dataset(f'{ds_dir}/{ds}.csv', pr_list=pr_list)

        if validation:
            X_train, y_train, X_test, y_test = train_test_split(X, y, n_test)
        else:
            X_train = X[:]
            y_train = y[:]
            X_test = X_train
            y_test = y_train
            
        # Итерация по моделям регрессии
        # models_list = []
        for name, regr in zip(names, regressors):
            one_model_row = dict()
                
            regr = regr.fit(X_train, y_train)
            y_predicted = regr.predict(X_test)
            
            # Коэффициенты уравнения (если есть)
            try:
                predictors_coef = {f: c for f, c in zip(pr_list, regr.coef_) if c != 0.0}
                predictors = ", ".join(predictors_coef.keys())
                equation = ' '.join(str(round(c, 2))+'*'+f for f, c in predictors_coef.items())
                equation = equation.replace(" -", "-")
                equation = equation.replace(" ", " + ")
                equation = equation.replace("-", " - ")
    
                one_model_row['Predictors'] = predictors
                one_model_row['Equations'] = equation
            except Exception:
                one_model_row['Predictors'] = ""
                one_model_row['Equations'] = ""

            # Название метода
            one_model_row['Method'] = name

            # Расчет показателей качества по методике
            
            sigma = get_sigma(y)
            delta_dop = get_delta_dop(sigma)
            s_forecast = get_s(y_test, y_predicted)
            
            
            # Критерий эффективности метода прогнозирования климатический S/sigma
            criterion_forecast = get_criterion(s_forecast, sigma)
            one_model_row['Criterion'] = round(criterion_forecast, 2)
            
            # Корреляционное отношение ro
            correlation_forecast = get_correlation_ratio(criterion_forecast)
            one_model_row['Correlation'] = round(correlation_forecast, 2)
            
            # Обеспеченность метода (оправдываемость) Pm
            pm = get_pm(y_test, y_predicted, delta_dop)
            one_model_row['Pm'] = round(pm, 2)

            # Model
            one_model_row['Model'] = regr

            # models_list.append(one_model_row)
            datasets_result[ds].append(one_model_row)

        # Сортировка результатов по каждому датасету
        datasets_result[ds].sort(key=lambda row: (row['Criterion'], -row['Correlation'], -row['Pm']))

        # datasets_result[ds[0]].append(models_list)
        #datasets_result.append(one_dataset_row)

        # Запись в .csv файл
        write_dataset_csv(datasets_result[ds], ds, fieldnames)

    return datasets_result

In [20]:
result = compare_models(validation=False, n_test=None)
print(result, sep='\n\n')

{'Неман-Белица': [{'Predictors': 's_2802, s_max, h, x, x1, x2, x3, x4, xs', 'Equations': '0.98*s_2802 + 0.31*s_max + 0.53*h + 1.61*x + 1.44*x1 - 0.24*x2 - 3.43*x3 + 12.16*x4 - 1.2*xs', 'Method': 'LinearRegression', 'Criterion': 0.51, 'Correlation': 0.86, 'Pm': 75.68, 'Model': LinearRegression()}, {'Predictors': 's_2802, s_max, h, x, x1, x2, x3, x4, xs', 'Equations': '1.05*s_2802 + 0.22*s_max + 0.51*h + 1.57*x + 1.4*x1 - 0.17*x2 - 0.38*x3 + 1.98*x4 - 1.15*xs', 'Method': 'Ridge', 'Criterion': 0.51, 'Correlation': 0.86, 'Pm': 75.68, 'Model': Ridge(alpha=10)}, {'Predictors': 's_2802, s_max, h, x, x1, x2, x3, x4, xs', 'Equations': '1.07*s_2802 + 0.19*s_max + 0.51*h + 1.49*x + 1.32*x1 - 0.1*x2 + 0.12*x3 + 0.3*x4 - 1.06*xs', 'Method': 'ElasticNet', 'Criterion': 0.51, 'Correlation': 0.86, 'Pm': 75.68, 'Model': ElasticNet(alpha=2.0, l1_ratio=0.1)}, {'Predictors': 's_2802, s_max, h, x, x1, x2, x3, xs', 'Equations': '1.07*s_2802 + 0.2*s_max + 0.51*h + 1.53*x + 1.36*x1 - 0.13*x2 + 0.21*x3 - 1.1*xs

In [21]:
# Получить лучшую модель по датасету с индексом [0] из результатов функции обучения и оценки моделей
dataset_name = 'Неман-Белица'
best_dataset_model = result[dataset_name][0]['Model']
verify_forecast(dataset_name, best_dataset_model, n_test=None)

In [22]:
new_prediction = []

### Формирование набора данных

In [23]:
#pr_list = ['s_2802', 's_max', 'h', 'x', 'x1', 'x2', 'x3', 'x4', 'xs']

In [24]:
pr_list = ['s_2802', 's_max', 'h', 'x', 'x1', 'x2', 'x3', 'xs']

In [25]:
X, y = get_river_dataset('data/Неман-Белица-2022.csv', pr_list=pr_list)

FileNotFoundError: [Errno 2] No such file or directory: 'data/Неман-Белица-2022.csv'

### Нормализация данных

In [ ]:
# Стандартизация
scaler = preprocessing.StandardScaler().fit(X)
X_train_st = scaler.transform(X)


### Модели регрессии

#### Линейная регрессия

In [ ]:
regr = linear_model.LinearRegression(fit_intercept=True)
regr = regr.fit(X, y)
y_predicted = regr.predict(X)
print(y_predicted)

### Вычисление статистики

In [ ]:
# Среднеквадратическое отклонение sigma
sigma = get_sigma('Неман-Белица')
print(f'Среднеквадратическое отклонение sigma = {sigma}')

# Допустимая погрешность прогноза delta_dop
delta_dop = get_delta_dop(sigma)
print(f'Допустимая погрешность прогноза sigma_dop = {delta_dop}')

#### Климат

In [ ]:
# Статистика максимального уровня
sum_level = get_sum(y)
avg_level = get_avg(y)
max_level = get_max(y)
min_level = get_min(y)
print(f'Сумма {sum_level}, Средний {avg_level}, Высший {max_level}, Низший {min_level}')

# Среднеквадратическая погрешность климатическая S
s_climate = get_s(y)
print(f'Среднеквадратическая погрешность климатическая: {s_climate}')

# Критерий эффективности метода прогнозирования климатический S/sigma
criterion_climate = get_criterion(s_climate, sigma)
print(f'Критерий эффективности метода прогнозирования климатический S/sigma = {criterion_climate}')

# Климатическая обеспеченность Pk
pk = get_pk(y, delta_dop)
print(f'Климатическая обеспеченность Pk, % = {pk}')

# Корреляционное отношение ro
ro_climate = get_correlation_ratio(criterion_climate)
print(f'Корреляционное отношение климатическое ro = {ro_climate}')

# Вероятная погрешность прогноза delta50
delta50_climate = get_delta50(s_climate)
print(f'Вероятная погрешность климатическая delta50 = {delta50_climate}')

#### Прогноз

In [ ]:
# Коэффициенты регрессии
print(regr.intercept_, regr.coef_)

# Среднеквадратическая погрешность климатическая S
s_forecast = get_s(y, y_predicted)
print(f'Среднеквадратическая погрешность прогноза: {s_forecast}')

# Критерий эффективности метода прогнозирования климатический S/sigma
criterion_forecast = get_criterion(s_forecast, sigma)
print(f'Критерий эффективности метода прогнозирования S/sigma = {criterion_forecast}')

# Обеспеченность метода (оправдываемость) Pm
pm = get_pm(y, y_predicted, delta_dop)
print(f'Обеспеченность метода (оправдываемость) Pm, % = {pm}')

# Корреляционное отношение ro
ro_forecast = get_correlation_ratio(criterion_forecast)
print(f'Корреляционное отношение ro = {ro_forecast}')

# Вероятная погрешность прогноза delta50
delta50_forecast = get_delta50(s_forecast)
print(f'Вероятная погрешность прогноза delta50 = {delta50_forecast}')

### Обучающий набор

In [ ]:
test_qnt = 5

In [ ]:
X_train = X[:-test_qnt]
X_test = X[-test_qnt:]

In [ ]:
y_train = y[:-test_qnt]
y_test = y[-test_qnt:]

## Этап нормализации